## Installation

In [ ]:
%pip install 'jax[cpu]'

## Jax 를 Numpy처럼 사용하기

사용자는 대부분 jax.numpy를 통해, 마치 numpy를 사용하듯 jax를 사용하게 됩니다.

In [1]:
import jax.numpy as jnp

In [2]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = jnp.arange(5.0)
print(selu(x))

[0.        1.05      2.1       3.1499999 4.2      ]


## `jax.jit()` 사용하기

`jax.jit()` 를 사용하면 XLA를 사용하여 함수를 컴파일 할 수 있습니다. 이를 통해 속도를 높일 수 있습니다.

jax 연산은 성능을 위해서 파이썬과 비동기적으로 돌아갑니다. 이러한 속성 때문에 jax를 사용하는 함수의 정확한 시간 측정을 위해서는 `block_until_ready()`를 사용해야 합니다.

In [3]:
from jax import random
key = random.key(0)
x = random.normal(key, (1000000, )) # jax에서는 랜덤한 수를 생성할 때마다 seed를 넣어줍니다.
%timeit selu(x).block_until_ready()  # runs on the CPU

1.63 ms ± 74.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


이번에는 `jax.jit()`를 사용하여 `selu` 함수를 실행해 보겠습니다:

In [4]:
from jax import jit

selu_jit = jit(selu)
_ = selu_jit(x) # 첫 실행은 jit을 컴파일하는 단계이므로, 최소 한번의 실행 이후 속도의 이점을 볼 수 있습니다. warm-up 단계로 생각해도 좋습니다.
%timeit selu_jit(x).block_until_ready()

632 µs ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## `jax.grad()`를 사용하여 미분하기

`jax.grad()`를 사용하면 함수의 도함수를 구할 수 있습니다. `jax.grad()` 는 [automatic differentiation](https://velog.io/@gypsi12/Auto-Differentiation자동미분-이란), 즉, 자동미분 방법을 사용합니다.

In [6]:
from jax import grad

def squared(x):
    return x**2

print(f"x^2 = {squared(4.0)}")               # y = x^2

grad_fn = grad(squared)
print(f"2x = {grad_fn(4.0)}")               # y = 2x

grad_grad_fn = grad(grad_fn)
print(f"2 = {grad_grad_fn(4.0)}")          # y =2


x^2 = 16.0
2x = 8.0
2 = 2.0


## `jax.vmap()`으로 백터화하기

`jax.vmap()`을 이용하여 동일한 연산을 병렬로 수행할 수 있습니다.
다만 `jax.vmap()`은 컴퓨터 단에서 멀티프로세싱을 한다는 말은 아닙니다. 스칼라끼리의 곱셈을 행렬연산으로 바꾸면 한 번의 계산으로 여러 곱셈을 효율적이고 빠르게 할 수 있듯이, 내부적으로 여러개의 연산을 한번의 계산으로 바꾸어 처리하는 방법입니다.
멀티프로세싱을 사용하는 방법은 `jax.pmap()`을 통해 가능합니다.

간단한 예제를 살펴보겠습니다. 해당 문제는 `jax.vmap()`를 굳이 사용하지 않아도 될 만큼 간단한 문제이지만, `jax.vmap()`의 작동 방식을 보여드리기 위해 사용하였습니다.

In [7]:
key1, key2 = random.split(key)
mat = random.normal(key1, (150, 100))
batched_x = random.normal(key2, (10, 100))

def apply_matrix(x):
  return jnp.dot(mat, x)

`mat` 와 `batched_x` 를 행렬곱하고 싶을 때, 행렬곱을 시행할 수 있는 가장 멍청한 방법은 다음일 것입니다:

In [9]:
def naively_batched_apply_matrix(v_batched):
  return jnp.stack([apply_matrix(v.T) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
456 µs ± 5.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


하지만 저희는 더 좋은 방법을 알고 있습니다. 한 행 한 행 따로 연산할 필요 없이, `batched_x`를 transpose(전치)하여 `mat`와 행렬곱을 하면 훨씬 효율적으로 같은 결과를 얻을 수 있습니다. `jnp.dot()` 함수를 통해 행렬곱을 시행할 수 있습니다:

In [10]:
@jit
def batched_apply_matrix(batched_x):
  return jnp.dot(batched_x, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched
16.4 µs ± 73.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


`jax.vmap()`은 사용자가 직접 배치 처리를 설계하지 않아도 되도록 도와줍니다:

In [11]:
from jax import vmap

@jit
def vmap_batched_apply_matrix(batched_x):
  return vmap(apply_matrix)(batched_x)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Auto-vectorized with vmap
22.8 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


지금 예제로 다루고 있는 문제는 `jnp.dot()` 만을 통해서 충분히 아름답게 해결할 수 있습니다. 하지만 내부 로직이 복잡해 질 수록 다양한 변수에 대하여 계산 오류가 생기지 않도록 신경써주어야 하고, 이는 코드를 난잡하게 만듭니다. 즉, `jnp.vmap()` 은 함수가 복잡해 질 수록 빛을 발합니다.

`jax.vmap()` 위에 `jax.jit()` 를 씌우던, `jax.jit()` 위에 `jax.vmap()` 를 씌우던 전혀 무관합니다.